In [13]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-15.0.1"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pyth

In [14]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer                                      
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pickle

with open('/content/drive/My Drive/Colab Notebooks/project/nlp/tokenizer_data.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [38]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

In [39]:
# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [40]:
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 43753
등장 빈도가 2번 이하인 희귀 단어의 수: 24337
단어 집합에서 희귀 단어의 비율: 55.623614380728185
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8715872104872904


In [41]:
vocab_size = total_cnt - rare_cnt + 2

In [42]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [44]:
model = load_model('/content/drive/My Drive/Colab Notebooks/project/nlp/model_1.h5')

In [45]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/project/nlp/ckpt/cp.ckpt"
model.load_weights(checkpoint_path)

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1941700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,059,077
Trainable params: 2,059,077
Non-trainable params: 0
_________________________________________________________________


In [47]:
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
max_len = 30

In [48]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

sentiment_predict('내용은 평이했으나 신혜선이 예쁘다. 특히 눈매가 참 예쁘다. 사랑스럽다.')

97.84% 확률로 긍정 리뷰입니다.



In [49]:
import requests
from bs4 import BeautifulSoup

In [50]:
def get_data(url):
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    for i, li in enumerate(lis):
        review_text = li.find('p').getText()  # span id = _filtered_ment_0
        review_text = review_text.replace("관람객","")
        review_text = review_text.strip()
        score = li.find('em').getText()

        # 간단하게 리뷰와 점수 프린트만 했습니다.
        print(score.strip(),review_text)
        sentiment_predict(review_text)

In [51]:
test_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=193194&type=after' # 테스트 (영화 도굴)
resp = requests.get(test_url)
html = BeautifulSoup(resp.content, 'html.parser')
# result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
total_count = 100 #int(result.replace(',', ''))

for i in range(1, int(total_count / 10) + 1):
    url = test_url + '&page=' + str(i)
    print('url: "' + url + '" is parsing....')
    get_data(url)

url: "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=193194&type=after&page=1" is parsing....
7 킬링타임용 영화 내용은 좀 뻔함 여운 없음
92.39% 확률로 부정 리뷰입니다.

10 이제훈 진짜 멋있당 ㅋㅋㅋ 웃을때마다 심쿵 ㅜㅜ
99.49% 확률로 긍정 리뷰입니다.

10 간만에 진짜 재밌게봄 ㅋㅋㅋㅋ 이제훈 귀ㅇㅕ워 ㅠㅠㅠㅠ
99.91% 확률로 긍정 리뷰입니다.

10 내용은 평이했으나 신혜선이 예쁘다. 특히 눈매가 참 예쁘다. 사랑스럽다.
97.84% 확률로 긍정 리뷰입니다.

10 간만에 진짜 많이 웃었네요 ㅋㅋ 배우들 케미가 끝내줍니다
99.68% 확률로 긍정 리뷰입니다.

10 우울했던 일상에 간만에 통쾌함을 안겨준 영화~
98.20% 확률로 긍정 리뷰입니다.

10 시간가는줄 모르고 봤네요? 진심 개꿀잼
93.21% 확률로 긍정 리뷰입니다.

10 이제훈 이런 캐릭터도 완젼 찰떡이네 ㅋㅋ 요즘시기에 유쾌하게 보기 좋은 영화
98.70% 확률로 긍정 리뷰입니다.

4 뻔한 스토리 박진감없고 그냥 그랬습니다. 리뷰 보면 다 재밌다고 난리던데 알바의심드네요.
99.97% 확률로 부정 리뷰입니다.

10 캐스팅도 좋고, 재미도 좋다
96.00% 확률로 긍정 리뷰입니다.

url: "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=193194&type=after&page=2" is parsing....
10 이제훈, 조우진, 짠희의 유쾌한 연기, 바로 이거임. 코로나 블루 날려버릴 한국판 오션스일레븐.
98.44% 확률로 긍정 리뷰입니다.

7 주인공 깐죽대다가도 진지할때가 있어야 하는데 끝날때까지 깐죽꺼리기만함
82.27% 확률로 긍정 리뷰입니다.

4 보는 내내 영화로는 스토리가 빈약하고 드라마로 나왔음 시청률 잘 나왔겠네 이생각 함. 전개가 너무 뻔해서 좀 지루했음.
99.53% 